專案: 製作通訊錄
目標: 以 Python 練習與 Postgres 資料庫互動，CRUD。

工具: 
- Postgres and Pgadmin on docker

思考點: 
- 原生 SQL（psycopg2） 還是 ORM (SQLAlchemy)? psycopg2 先練習寫 SQL 語句。

前端: 簡易通訊錄表格
Flask RestAPI: 先練習以 HTTP 方式操作通訊錄
後端: Postgres database

proj-address-book/
  - requirements.txt
    Flask==3.0.3
    psycopg2-binary==2.9.9
    flask-cors==4.0.0
  - app.py
  - front/index.html
    

CRUD: create, retrive, update, delete (新增、讀取、更新、刪除)
REST: POST, GET, PUT, DELETE (建立新物件、讀取物件資料、更新物件的資料、刪除資料)

In [ ]:
pyenv local 3.10.7
pyenv exec python -m venv .venv
source .venv/bin/activate

In [ ]:
pip install -r requirements.txt

In [ ]:
vim addr-book-app.py

#從 flask 函式庫或稱模組，匯入 Flask, request, jsonify 功能
#from psycopg2.extras import RealDictCursor 是 postgreSQL 連線，在查詢結果以 dict 而不是 tuple 的形式回傳，增加可讀性，也方便轉成 JSON 給 API 回傳。
from flask import Flask, request, jsonify
from flask_cors import CORS
import psycopg2-binary
from psycopg2.extras import RealDictCursor
import os


#建立 Flask 應用實例，後續所有的路由、設定都會掛在這個物件上
app = Flask(__name__)

#開啟 CORS 
#瀏覽器為了安全，預設禁止前端 JS 存取跟當前網頁不同來源的資源，避免惡意網站亂用你的登入身分取資料。
#測試可以 CORS(app) ，允許所有來源
CORS(app, origins=["https://localhost:8000", "http://127.0.0.1:5500"])

#DB 連線設定
#去系統環境變數找 DB_HOST 的值
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "postgres")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASS = os.getenv("DB_PASS", "postgres")

#定義路由
@app.route("/", method=['GET', 'POST'])
def home():
    return "Hello World"

#如果 if __name__ == "__main__" 成立，執行 app.run(debug=True)
#debug=True 改了程式檔會重啟伺服器，debug mode 除錯模式，發生錯誤時瀏覽器會顯示詳細錯誤
if __name__ == "__main__"
    app.run(debug=True)
